**Upload your files here.**

In [ ]:
from google.colab import files
uploads = files.upload()

# Met Oddy Test CNN

This code will prepare the model, it only has to be run once.

### 1.1 Process Inputs

**Run this if you want to clear old files before uploading more.**

In [78]:
# Add folder for input images
!rm -rf input_images
!mkdir input_images
# Make directory for outputted images
!rm -rf result_directory
!mkdir result_directory

In [79]:
# Sets up the ability to ask for files
import os

# Asks you to upload a file
for i in uploads.keys():
  if ' ' in i:
    os.rename("/content/"+i, "/content/"+i.replace(" ","_"))
  i = i.replace(" ","_")
  file = "/content/"+i
  !mv $file "/content/input_images"
print("Done moving!")

Done moving!


The following Python notebook will run an inputted batch of images through a trained convolutional neural network to predict the corrosion level in Oddy Test images. There a few things you will need to do to set up this notebook. 

Click on the play button on the left hand side of the code boxes to run the code.

## Prepare files

### 1.2 Get checkpoint files

In [4]:
!wget https://raw.githubusercontent.com/fredthefish/OddyTest/main/checkpoint -P MetCheckpoint
!wget https://raw.githubusercontent.com/fredthefish/OddyTest/main/ckpt-49.index -P MetCheckpoint
!wget https://raw.githubusercontent.com/fredthefish/OddyTest/main/ckpt-49.data-00000-of-00001 -P MetCheckpoint

checkpoint_path = '/content/MetCheckpoint/ckpt-49'

--2022-08-18 17:56:44--  https://raw.githubusercontent.com/fredthefish/OddyTest/main/checkpoint
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 706 [text/plain]
Saving to: ‘MetCheckpoint/checkpoint’

checkpoint          100%[===================>]     706  --.-KB/s    in 0s      

2022-08-18 17:56:45 (29.2 MB/s) - ‘MetCheckpoint/checkpoint’ saved [706/706]

--2022-08-18 17:56:45--  https://raw.githubusercontent.com/fredthefish/OddyTest/main/ckpt-49.index
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 114788 (112K) [application/oct

## 2. Prepare the Model

Prepares the model. This code only has to be run once. Note that it may take 10 minutes to complete.

### 2.1 Install TensorFlow and Object Detection API


In [6]:
# Install Tensorflow latest version
# !pip install --upgrade tensorflow
!pip install -U --pre tensorflow=="2.9.1"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 511.7 MB 6.2 kB/s 
     |████████████████████████████████| 5.8 MB 49.5 MB/s 
     |████████████████████████████████| 438 kB 49.9 MB/s 
     |████████████████████████████████| 1.6 MB 49.3 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.8.0
    Uninstalling tensorflow-estimator-2.8.0:
      Successfully uninstalled tensorflow-estimator-2.8.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.8.0
    Uninstalling tensorboard-2.8.0:
      Successfully uninstalled tensorboard-2.8.0
  Attempting uninstall: keras
    Found existing installation: keras 2.8.0
    Uninstalling keras-2.8.0:
      Successfully uninstalled keras-2.8.0
  Attempting uninstall: gast
    Found existing installation: gast 0.5.3
    Uninstalling gast-0.5.3:
      Successfully uninstalled gast-0.

In [7]:
# Need updated package for libcudnn8 so that step 7 doesn't trigger an error
!apt install --allow-change-held-packages libcudnn8

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  libcudnn8-dev
The following held packages will be changed:
  libcudnn8
The following packages will be upgraded:
  libcudnn8 libcudnn8-dev
2 upgraded, 0 newly installed, 0 to remove and 18 not upgraded.
Need to get 704 MB of archives.
After this operation, 2,625 MB disk space will be freed.
Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  libcudnn8-dev 8.5.0.96-1+cuda11.7 [356 MB]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  libcudnn8 8.5.0.96-1+cuda11.7 [347 MB]
Fetched 704 MB in 23s (30.3 MB/s)
(Reading database ... 155676 files and directories currently installed.)
Preparing to unpack .../libcudnn8-dev_8.5.0.96-1+cuda11.7_amd64.deb ..

In [8]:
# Import tensorflow
import tensorflow as tf
# Check tensorflow version
# print(tf.__version__)

Install the Tensorflow 2 [Object Detection API](https://github.com/tensorflow/models/tree/master/research/object_detection):

In [9]:
# uncomment the next line if you want to delete an existing models directory
# !rm -rf ./models/

# clone the Tensorflow Model Garden
!git clone --depth 1 https://github.com/tensorflow/models.git

Cloning into 'models'...
remote: Enumerating objects: 3451, done.
remote: Counting objects: 100% (3451/3451), done.
remote: Compressing objects: 100% (2891/2891), done.
remote: Total 3451 (delta 891), reused 1418 (delta 503), pack-reused 0
Receiving objects: 100% (3451/3451), 46.85 MiB | 21.72 MiB/s, done.
Resolving deltas: 100% (891/891), done.
Checking out files: 100% (3125/3125), done.


In [10]:
# install the Object Detection API
!cd models/research/ && protoc object_detection/protos/*.proto --python_out=. && cp object_detection/packages/tf2/setup.py . && python -m pip install .

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing /content/models/research
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
     |████████████████████████████████| 10.9 MB 9.7 MB/s 
     |████████████████████████████████| 352 kB 46.9 MB/s 
     |████████████████████████████████| 2.1 MB 53.3 MB/s 
     |████████████████████████████████| 25.9 MB 1.2 MB/s 
     |████████████████████████████████| 67 kB 8.1 MB/s 
     |████████████████████████████████| 636 kB 67.5 MB/s 
     |████████████████████████████████| 4.6 MB 56.2 MB/s 
     |████████████████████████████████| 99 kB 12.1 MB/s 
     |██

### 2.2 Import Packages

In [11]:
!pip install dfply
from dfply import *

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 612 kB 4.8 MB/s 


In [12]:
import matplotlib
import matplotlib.pyplot as plt

import os
import random
import zipfile
import io
import scipy.misc
import numpy as np
import pandas as pd

import glob
import imageio
from six import BytesIO
from PIL import Image, ImageDraw, ImageFont
from IPython.display import display, Javascript
from IPython.display import Image as IPyImage

In [13]:
# import the label map utility module
from object_detection.utils import label_map_util

# import module for reading and updating configuration files.
from object_detection.utils import config_util

# import module for visualization. use the alias `viz_utils`
from object_detection.utils import visualization_utils as viz_utils

# import module for building the detection model
from object_detection.builders import model_builder

# import module for utilities in Colab
from object_detection.utils import colab_utils

### 2.3 Import and Clean Label Data
We need to grab the Label Map for the classes from GitHub (StringIntLabelMap.pbtxt). We'll then define a few dictionaries for the 9 classes in the model. 

In [14]:
# Load label map from file
# Function found here: https://github.com/tensorflow/models/blob/master/research/object_detection/utils/label_map_util.py
!wget https://raw.githubusercontent.com/fredthefish/OddyTest/main/StringIntLabelMap.pbtxt -P dir
label_map = label_map_util.load_labelmap('/content/dir/StringIntLabelMap.pbtxt')

--2022-08-18 18:00:39--  https://raw.githubusercontent.com/fredthefish/OddyTest/main/StringIntLabelMap.pbtxt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 360 [text/plain]
Saving to: ‘dir/StringIntLabelMap.pbtxt’

StringIntLabelMap.p 100%[===================>]     360  --.-KB/s    in 0s      

2022-08-18 18:00:39 (14.9 MB/s) - ‘dir/StringIntLabelMap.pbtxt’ saved [360/360]



In [15]:
# Convert to dictionary
label_dict = label_map_util.get_label_map_dict(label_map,use_display_name=True)
label_dict

{'Ag-P': 1,
 'Ag-T': 2,
 'Ag-U': 3,
 'Cu-P': 4,
 'Cu-T': 5,
 'Cu-U': 6,
 'Pb-P': 7,
 'Pb-T': 8,
 'Pb-U': 9}

In [16]:
# define a dictionary describing the corrosion classes
category_index = {
    1 : {
        'id'  : 1, 
        'name': 'Ag-P'
    },
    2 : {
        'id'  : 2,
        'name': 'Ag-T'
    },
    3 : {
        'id'  : 3,
        'name': 'Ag-U'
    },
    4 : {
        'id'  : 4,
        'name': 'Cu-P'
    },
    5 : {
        'id'  : 5,
        'name': 'Cu-T'
    },
    6 : {
        'id'  : 6,
        'name': 'Cu-U'
    },
    7 : {
        'id'  : 7,
        'name': 'Pb-P'
    },
    8 : {
        'id'  : 8,
        'name': 'Pb-T'
    },
    9 : {
        'id'  : 9,
        'name': 'Pb-U'
    }
}

In [17]:
# Testing grabbing the categories
# print(category_index[1])

In [18]:
# Specify the number of classes that the model will predict
num_classes = 9

### 2.4 Configure the CNN Model & Build with Model Checkpoint

Gets the model config from github, builds the model, and imports the trained model checkpoint.

In [19]:
# Clears old models
tf.keras.backend.clear_session()

# EfficientDet V3:
!wget https://raw.githubusercontent.com/fredthefish/OddyTest/main/eff_det_d1_pipeline_5.config -P dir
pipeline_config = '/content/dir/eff_det_d1_pipeline_5.config'

--2022-08-18 18:00:39--  https://raw.githubusercontent.com/fredthefish/OddyTest/main/eff_det_d1_pipeline_5.config
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4435 (4.3K) [text/plain]
Saving to: ‘dir/eff_det_d1_pipeline_5.config’

eff_det_d1_pipeline 100%[===================>]   4.33K  --.-KB/s    in 0s      

2022-08-18 18:00:39 (74.0 MB/s) - ‘dir/eff_det_d1_pipeline_5.config’ saved [4435/4435]



In [20]:
# Load the configuration file into a dictionary
configs = config_util.get_configs_from_pipeline_file(pipeline_config)

In [21]:
# Read in the object stored at the key 'model' of the configs dictionary
model_config = configs['model']

In [22]:
# Use the model_builder build function from the config above
detection_model = model_builder.build(model_config = model_config, is_training = True)

In [23]:
detection_model

Load in the checkpoint from Google Drive.

Note: do not point the checkpoint path at ckpt-49.index because we also need the checkpoint data file.

In [24]:
#!wget https://raw.githubusercontent.com/fredthefish/OddyTest/main/ckpt-49.index -P dir
# checkpoint_path = '/content/drive/MyDrive/Dissertation/models_workspace/MetCheckpoint/ckpt-49'

# Define a checkpoint
checkpoint = tf.compat.v2.train.Checkpoint(model=detection_model)

# Restore the checkpoint to the checkpoint path
checkpoint.restore(checkpoint_path).expect_partial()

Run a dummy image through the model so that variables are created.

In [25]:
# use the detection model's `preprocess()` method and pass a dummy image
tmp_image, tmp_shapes = detection_model.preprocess(tf.zeros([1, 640, 640, 3]))

# run a prediction with the preprocessed image and shapes
tmp_prediction_dict = detection_model.predict(tmp_image, tmp_shapes)

# postprocess the predictions into final detections
tmp_detections = detection_model.postprocess(tmp_prediction_dict, tmp_shapes)

print('Weights restored!')

Weights restored!


In [26]:
# Check on the shape of the model:
# tmp_shapes

### 2.5 Define Functions 

Processing an Image with the Neural Network, Cleaning the Outputs, and Saving Labelled Images


**plot_detections** for plotting bounding boxes on images

In [27]:
  def plot_detections(image_np,
                    boxes,
                    classes,
                    scores,
                    category_index,
                    figsize=(12, 16),
                    image_name=None):
    """Wrapper function to visualize detections.

    Args:
    image_np: uint8 numpy array with shape (img_height, img_width, 3)
    boxes: a numpy array of shape [N, 4]
    classes: a numpy array of shape [N]. Note that class indices are 1-based,
          and match the keys in the label map.
    scores: a numpy array of shape [N] or None.  If scores=None, then
          this function assumes that the boxes to be plotted are groundtruth
          boxes and plot all boxes as black with no classes or scores.
    category_index: a dict containing category dictionaries (each holding
          category index `id` and category name `name`) keyed by category indices.
    figsize: size for the figure.
    image_name: a name for the image file.
    """
    
    image_np_with_annotations = image_np.copy()
    
    viz_utils.visualize_boxes_and_labels_on_image_array(
        image_np_with_annotations,
        boxes,
        classes,
        scores,
        category_index,
        use_normalized_coordinates=True,
        max_boxes_to_draw=30,
        min_score_thresh=0.3,
        line_thickness = 10)
    
    if image_name:
        plt.imsave(image_name, image_np_with_annotations)
    
    else:
        plt.imshow(image_np_with_annotations)

**load_image_into_numpy_array** turns a jpg into a numpy array

In [28]:
def load_image_into_numpy_array(path):
    """Load an image from file into a numpy array.
    Shape: (height, width, channels), where channels=3 for RGB.

    Args: path - a file path.
    Returns: uint8 numpy array with shape (img_height, img_width, 3)
    """
    
    img_data = tf.io.gfile.GFile(path, 'rb').read()
    image = Image.open(BytesIO(img_data))
    (im_width, im_height) = image.size
    
    return np.array(image.getdata()).reshape(
        (im_height, im_width, 3)).astype(np.uint8)

**load_image_set** loads a folder of images into variables for numpy arrays and tensors

In [29]:
def load_image_set(image_dir):
    """Load a folder of images, converts to numpy arrays, and then converts to tensors.
    Args: 
      image_dir - a path to folder of training, validation, or test images. 
    Returns:
      image_nps - numpy arrays for images
      image_tensors - image tensors
      files - file names from image_dir
    """
    # Get a list of the files in the image folder
    files = os.listdir(image_dir)
    # Starting an empty list for the npy arrays
    image_nps = []
    # List for image tensors
    image_tensors = []
    # Iterate and load each image in the file
    for idx, file in enumerate(files):
        # define the path (string) for each image
        image_path = os.path.join(image_dir,file)
        # load image into numpy array and append to list
        image_np = load_image_into_numpy_array(image_path)
        image_nps.append(image_np)
        # convert training image to tensor, add batch dimension, and add to list
        image_tensors.append(tf.expand_dims(tf.convert_to_tensor(image_np, dtype=tf.float32), axis=0))
        # Give a loading update every 5 images
        if idx % 5 == 0:
          print('Loading',str(idx),':',file)
    print('Done Loading and Saving!')
    return image_nps, image_tensors, files

**detect** runs an image through the CNN and outputs the detections

In [30]:
# Uncomment this decorator if you want to run inference eagerly
@tf.function
def detect(input_tensor):
    """Run the neural network on an input image.

    Args:
    input_tensor: A [1, height, width, 3] Tensor of type tf.float32.
      Note that height and width can be anything since the image will be
      immediately resized according to the needs of the model within this
      function.

    Returns:
    3 Tensors (`detection_boxes`, `detection_classes`, and `detection_scores`).
    """
    preprocessed_image, shapes = detection_model.preprocess(input_tensor)
    prediction_dict = detection_model.predict(preprocessed_image, shapes)
    # use the detection model's postprocess() method to get the the final detections
    detections = detection_model.postprocess(prediction_dict, shapes) 

    return detections

**detect_extract** gets the predicted classes, scores, and bounding boxes from the CNN detections

In [31]:
def detect_extract(input_tensor):
    ''' Function to extract the predicted classes, scores, and bounding boxes'''

    # Run neural network on image
    detections = detect(input_tensor)

    # Get predicted classes, scores, and boxes
    class_array = detections['detection_classes'][0].numpy().astype('int') + 1
    det_scores = detections['detection_scores'][0].numpy()
    det_boxes = detections['detection_boxes'][0].numpy()

    return class_array, det_scores, det_boxes

**intersection_over_union** calculates the Intersection Over Union (IOU) for two bounding boxes

In [32]:
def intersection_over_union(pred_box, true_box):
    # Split the predicted box array into separate values: (ymin, xmin, ymax, xmax)
    ymin_pred, xmin_pred, ymax_pred, xmax_pred = np.split(pred_box, 4)
    ymin_true, xmin_true, ymax_true, xmax_true = np.split(true_box, 4)

    smoothing_factor = 1e-10

    xmin_overlap = np.maximum(xmin_pred, xmin_true)
    xmax_overlap = np.minimum(xmax_pred, xmax_true)
    ymin_overlap = np.maximum(ymin_pred, ymin_true)
    ymax_overlap = np.minimum(ymax_pred, ymax_true)

    pred_box_area = (xmax_pred - xmin_pred) * (ymax_pred - ymin_pred)
    true_box_area = (xmax_true - xmin_true) * (ymax_true - ymin_true)

    overlap_area = np.maximum((xmax_overlap - xmin_overlap), 0)  * np.maximum((ymax_overlap - ymin_overlap), 0)
    union_area = (pred_box_area + true_box_area) - overlap_area
    
    iou = (overlap_area + smoothing_factor) / (union_area + smoothing_factor)

    return iou

**chooseBoxes** reduces the number of bounding box predictions to just the highest scoring ones that don't overlap

In [33]:
def chooseBoxes(image_tensor):
  ''' This function finds the highest scoring boxes that do not overlap with eachother for plotting 
  Args: image_tensors - one image in tensor format
  Returns: reduced lists of predicted classes, scores, and bounding boxes
  '''

  # Run the image through the neural network to get predictions
  det_classes, det_scores, det_boxes = detect_extract(image_tensor)

  # Create a list of boxes to keep for plotting
  keep_list = [0]
  # Iterate over all the predicted boxes
  for i in range(len(det_scores)):
      # Ignore any boxes with scores of 0
      if det_scores[i] == 0:
        break
      # The first box (i = 0) will always be included because it has the highest score
      if i == 0 :
        continue
      # Create list to store the IOUs of box i
      ious = []
      # Need to find intersections with all the boxes currently being 'kept' for display on images
      for j in keep_list:
        iou = intersection_over_union(det_boxes[i],det_boxes[j])
        # print('Box', str(i),'has iou with box',str(j),': ',iou)
        ious.append(iou)
      # Sorts the IOUs to find the highest IOU between box i and the boxes in the keep list
      ious.sort(reverse = True)
      # If the largest IOU is less than 0.2 (poor fit with all other boxes), add this box to the list
      if ious[0] < 0.2:
          keep_list.append(i)
          # print('Added box', str(i),'to keep list')
      
  # Return only the necessary boxes
  return det_classes[keep_list], det_scores[keep_list], det_boxes[keep_list]

**scanImage** runs an image through the CNN and saves an image of the best bounding boxes

In [34]:
def scanImage(image_nps, image_tensors, files, i):
  ''' Runs an image through the CNN, finds only the best bounding boxes (chooseBoxes function),
  and plots and saves the image labelled with predictions.
  Args: 
  image_nps - numpy image stack.
  image_tensors - tensor image stack
  files - file names of images for saving. 
  i - number of image
  Returns: the predicted classes, bounding boxes, and scores (class_array, det_boxes, det_scores)
  '''

  # Get the predicted bounding boxes from the neural network that don't overlap
  class_array, det_scores, det_boxes = chooseBoxes(image_tensors[i])

  # Define the figure size
  fig = plt.figure(figsize=(80, 50))
  # Use the plot_detections function to draw the ground truth boxes
  plot_detections(
      image_nps[i],
      det_boxes,
      class_array,
      det_scores,
      category_index = category_index,
  )
  #Save 
  plt.savefig("/content/result_directory/"+files[i].replace("jpg", "png"))
  plt.close(fig)

  # Print an update every 5 images, and for the first and last images.
  if (i == 0) or ((i+1) % 5 == 0) or ((i+1) == len(image_nps)):
    print(str(i+1)+"/"+str(len(image_nps))+": content/result_directory/"+files[i].replace("jpg", "png"))

  return class_array, det_boxes, det_scores

**findMetals** creates a dataframe of the predicted classes (2 per metal type)

In [35]:
def findMetals(pred_boxes, class_array, files, i):
  '''
  Args: 
  pred_boxes: dataframe for the predictions
  class_array: array of classes for bounding boxes
  files - file names of images for saving. 
  i - number of image

  Returns:
  Updated pred_boxes dataframe with the classes from 1-9 or 0 for NA values
  '''

  # Add filename to dataframe
  predBoxes.at[i,'FileName']  = files[i].replace('jpg','png')

  # Finding silver predictions and adding to dataframe
  Ag = class_array[class_array <= 3]
  if len(Ag) == 0:
    # If no silver predicted, leave as 0's and give warning message
    print('No silver coupons detected in file',files[i])
  elif len(Ag) == 1:
    # If only one silver, add to first column
    predBoxes.iloc[i,1] = Ag[0]
  else:
    # If two or more silver, just grab the first two
    predBoxes.iloc[i,1:3] = Ag[0:2]
  # Finding copper predictions and adding to dataframe
  Cu = class_array[(class_array > 3) & (class_array <= 6)]
  if len(Cu) == 0:
    print('No copper coupons detected in file',files[i])
  elif len(Cu) == 1:
    predBoxes.iloc[i,3]  = Cu[0]
  else:
    predBoxes.iloc[i,3:5] = Cu[0:2]
  # Finding lead predictions and adding to dataframe
  Pb = class_array[class_array >= 7]
  if len(Pb) == 0:
    print('No lead coupons detected in file',files[i])
  elif len(Pb) == 1:
    predBoxes.iloc[i,5]  = Pb[0]
  else:
    predBoxes.iloc[i,5:7] = Pb[0:2]

  return predBoxes

# Get Model Output

This code will give you the output files.

## 3.1 Convert Images, Process Them, and Output Results

This section processes the uploaded through the neural network, and saves the predictions both as images and a csv file.

It then runs the scanImage function to process the image through the neural network. The chooseBoxes function is called to prevent overlapping bounding boxes from being presented.

The scanImage function will save the Oddy test images with the bounding boxes. It will make the files in the result_directory folder.

### 3.1.1 Set Up for Images and Dataframe for Classes

This section starts by converting the uploaded images to numpy and tensor files.

In [80]:
# Gets npy and tensors versions of images
image_nps, image_tensors, files = load_image_set("/content/input_images")

Loading 0 : 620px-Sensuede2050_422_20141205.jpg
Done Loading and Saving!


In [81]:
# Gets a count of the number of uploaded images
count = len(image_tensors)
count

5

Create a dataframe to store the predicted classes.

In [82]:
# Creating empty data frame to store the classes outputted by the model for csv file
z = [0] * count
blank_row = {'FileName': ['']*count, 'Ag1': z, 'Ag2': z, 'Cu1': z, 'Cu2': z, 'Pb1': z, 'Pb2': z}
predBoxes = pd.DataFrame(blank_row)

### 3.1.2 Process Your Images
Keep an eye on the logs below, as they may show when the model has made errors, such as not detecting any lead coupons for a particular image.

In [83]:
# Iterate over each image to save class info and labelled images
for i in range(count):
   # Run the scan image function to save images with the bounding boxes
   class_array, det_boxes, det_scores = scanImage(image_nps, image_tensors, files, i)
   # Run the findMetals function to get the class data formatted as a dataframe
   predBoxes = findMetals(predBoxes, class_array, files, i)

1/5: content/result_directory/620px-Sensuede2050_422_20141205.png
No silver coupons detected in file 560px-Sensuederain2004_418_20141205.jpg


5/5: content/result_directory/608px-Sensuede2064_424_20141205.png


### 3.1.3 Save CSV file of predictions and download zipped folder of images and csv file

Depending on how many images you uploaded, this could take a few minutes to download the zip folder.

In [84]:
# A dictionary to convert class integers to P/T/U ratings with additional NA rating
reverse_dict = {0:'N/A',1:'Ag-P', 2:'Ag-T', 3:'Ag-U', 4:'Cu-P', 5:'Cu-T', 6:'Cu-U', 7:'Pb-P', 8:'Pb-T', 9:'Pb-U'}
# Mapping classes for ground truth and predicted classes
for i in [1,2,3,4,5,6]:
  # Map the label dictionary to a column to populate the corresponding class integer values
  # https://kanoki.org/2019/04/06/pandas-map-dictionary-values-with-dataframe-columns/
  predBoxes.iloc[:,i] = predBoxes.iloc[:,i].map(reverse_dict)

In [85]:
predBoxes = predBoxes.reindex(columns=['FileName','Cu1','Cu2','Ag1','Ag2','Pb1','Pb2'])
predBoxes

,FileName,Cu1,Cu2,Ag1,Ag2,Pb1,Pb2
0,620px-Sensuede2050_422_20141205,Cu-P,N/A,Ag-T,Ag-T,Pb-U,Pb-U
1,593px-Sensuede9_420_20141205,Cu-U,Cu-U,Ag-T,Ag-U,Pb-U,N/A
2,560px-Sensuederain2004_418_20141205,Cu-U,Cu-U,N/A,N/A,Pb-U,Pb-P
3,608px-Sensuede2081_426_20141205,Cu-U,Cu-U,Ag-U,N/A,Pb-U,Pb-U
4,608px-Sensuede2064_424_20141205,Cu-U,Cu-U,Ag-T,Ag-T,Pb-U,Pb-U


In [86]:
predBoxes.to_csv('result_directory/OddyTestPredictions.csv', index = False, index_label=False)  

In [88]:
from google.colab import files
!rm /content/result_directory.zip
!zip -r /content/result_directory.zip /content/result_directory/
files.download("/content/result_directory.zip")

  adding: content/result_directory/ (stored 0%)
  adding: content/result_directory/560px-Sensuederain2004_418_20141205.png (deflated 16%)
  adding: content/result_directory/OddyTestPredictions.csv (deflated 56%)
  adding: content/result_directory/620px-Sensuede2050_422_20141205.png (deflated 15%)
  adding: content/result_directory/608px-Sensuede2081_426_20141205.png (deflated 15%)
  adding: content/result_directory/593px-Sensuede9_420_20141205.png (deflated 15%)
  adding: content/result_directory/608px-Sensuede2064_424_20141205.png (deflated 15%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>